In [1]:
# debug backtester
from backtester import Backtester, read_trader_log
from parse_log import parse_log
from datamodel import Listing
from importlib import reload

import trader_kin
reload(trader_kin)

Trader = trader_kin.Trader
Product = trader_kin.Product

def calculate_kelp_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_resin_fair(order_depth):
    return 10000

listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 20,
    'KELP': 20
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_resin_fair,
    "KELP": calculate_kelp_fair
}

market_data, trade_history = read_trader_log('logs/empty_submission.log')

backtest_dir = 'backtests/trader_kin.log'

trader = Trader()

backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir)
backtester.run()

In [2]:
backtester.pnl

{'RAINFOREST_RESIN': 1394, 'KELP': 0.0}

In [88]:
sandbox, market, trades = parse_log(backtest_dir)

In [89]:
resin = market_data[market_data['product'] == 'RAINFOREST_RESIN']
resin

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,RAINFOREST_RESIN,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,2.0
3,-1,100,RAINFOREST_RESIN,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,2.0
4,-1,200,RAINFOREST_RESIN,9995,20,NaN,NaN,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10000.0,2.0
7,-1,300,RAINFOREST_RESIN,9996,2,9995.0,29.0,NaN,NaN,10004,2,10005.0,29.0,NaN,NaN,10000.0,2.0
8,-1,400,RAINFOREST_RESIN,9996,2,9995.0,25.0,NaN,NaN,10004,2,10005.0,25.0,NaN,NaN,10000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,-1,199500,RAINFOREST_RESIN,10002,1,9996.0,1.0,9995.0,22.0,10004,1,10005.0,22.0,NaN,NaN,10003.0,1394.0
3993,-1,199600,RAINFOREST_RESIN,9996,1,9995.0,21.0,NaN,NaN,10002,7,10004.0,1.0,10005.0,21.0,9999.0,1394.0
3994,-1,199700,RAINFOREST_RESIN,9998,8,9996.0,2.0,9995.0,25.0,10004,2,10005.0,25.0,NaN,NaN,10001.0,1394.0
3996,-1,199800,RAINFOREST_RESIN,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,1394.0


In [90]:
trades[(trades['buyer'] == 'SUBMISSION') | (trades['seller'] == 'SUBMISSION')]

,timestamp,buyer,seller,symbol,currency,price,quantity
0,0,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
1,0,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,0
6,1200,SUBMISSION,,RAINFOREST_RESIN,SEASHELLS,9998,1
7,1300,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
10,1800,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
...,...,...,...,...,...,...,...
2083,197400,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
2087,198500,SUBMISSION,,RAINFOREST_RESIN,SEASHELLS,9998,10
2091,198900,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
2106,199200,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10000,5


In [91]:
resin[(resin['bid_price_1'] >= 10000) | (resin['ask_price_1'] <= 10000)]

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,RAINFOREST_RESIN,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,2.0
12,-1,600,RAINFOREST_RESIN,10000,5,9995.0,25.0,NaN,NaN,10005,25,NaN,NaN,NaN,NaN,10002.5,2.0
24,-1,1200,RAINFOREST_RESIN,9995,21,NaN,NaN,NaN,NaN,9998,1,10005.0,21.0,NaN,NaN,9996.5,4.0
27,-1,1300,RAINFOREST_RESIN,10002,1,9996.0,1.0,9995.0,30.0,10004,1,10005.0,30.0,NaN,NaN,10003.0,6.0
36,-1,1800,RAINFOREST_RESIN,10002,1,9996.0,1.0,9995.0,22.0,10004,1,10005.0,22.0,NaN,NaN,10003.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,-1,197400,RAINFOREST_RESIN,10002,1,9995.0,20.0,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10003.5,1370.0
3970,-1,198500,RAINFOREST_RESIN,9995,22,NaN,NaN,NaN,NaN,9998,10,10005.0,22.0,NaN,NaN,9996.5,1390.0
3978,-1,198900,RAINFOREST_RESIN,10002,1,9998.0,8.0,9996.0,1.0,10004,1,10005.0,26.0,NaN,NaN,10003.0,1392.0
3985,-1,199200,RAINFOREST_RESIN,10000,5,9995.0,22.0,NaN,NaN,10002,9,10005.0,22.0,NaN,NaN,10001.0,1392.0


In [3]:
# benchmark
import trader_v2_benchmark
reload(trader_v2_benchmark)

backtest_dir_2 = 'backtests/trader_v2_benchmark.log'

trader_2 = trader_v2_benchmark.Trader()

backtester_2 = Backtester(trader_2, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir)
backtester_2.run()

In [4]:
backtester_2.pnl

{'RAINFOREST_RESIN': 3010, 'KELP': 0.0}

In [94]:
sandbox_2, market_2, trades_2 = parse_log(backtest_dir)

In [98]:
resint2 = trades_2[(trades_2['buyer'] == 'SUBMISSION') | (trades_2['seller'] == 'SUBMISSION')]
resint2 = resint2[resint2['symbol'] == 'RAINFOREST_RESIN']
resint2

,timestamp,buyer,seller,symbol,currency,price,quantity
0,0,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
1,0,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,0
9,1100,SUBMISSION,,RAINFOREST_RESIN,SEASHELLS,9997,2
11,1200,SUBMISSION,,RAINFOREST_RESIN,SEASHELLS,9998,1
12,1300,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
...,...,...,...,...,...,...,...
2653,198900,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10002,1
2658,199000,SUBMISSION,,RAINFOREST_RESIN,SEASHELLS,9997,2
2670,199200,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10000,5
2677,199400,,SUBMISSION,RAINFOREST_RESIN,SEASHELLS,10003,1
